### Pydantic

In [10]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY","")
model = init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002445AD6E960>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002445AE4EC60>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="The year the movie was released")
    director: str = Field(description="The director of the movie")
    rating: float = Field(description="The rating of the movie on a scale of 1 to 10") 

In [12]:
model_with_structure = model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002445AD6E960>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002445AE4EC60>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'The year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The rating of the movie on a scale of 1 to 10', 'type': 'number'}}, 'required': ['

In [14]:
model_with_structure.invoke("Provide the title, release year, director, and rating of the movie Inception.")

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

### Message Output along with Parsed structure

In [16]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The rating of the movie on a scale of 1 to 10") 
    
model_with_structure = model.with_structured_output(Movie, include_raw=True)
model_with_structure.invoke("Provide the title, release year, director, and rating of the movie Inception.")

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the title, release year, director, and rating of the movie Inception. Let me check the tools provided. There\'s a function called Movie that requires those exact parameters: title, year, director, and rating. So I need to call that function with the details about Inception.\n\nFirst, I need to recall the information about Inception. The title is "Inception". It was released in 2010. The director is Christopher Nolan. As for the rating, it\'s generally well-rated; I think it\'s around 8.8 on IMDb. But since the user didn\'t specify a particular rating source, I\'ll go with a commonly known rating. \n\nNow, I need to structure the function call with these parameters. The required fields are title, year, director, and rating. All are present here. The function expects a JSON object with those properties. Let me make sure the data types are correct: title and director are strings, year is an

### NESTED STRUCTURE

In [17]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str
    role: str
    
class MovieDetails(BaseModel):
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The rating of the movie on a scale of 1 to 10") 
    actors: list[Actor] = Field(..., description="A list of actors in the movie")
    budget: float | None = Field(None, description="The budget of the movie in millions of dollars")
    
model_with_structure = model.with_structured_output(MovieDetails)
model_with_structure.invoke("Provide the details about inception movie.")

MovieDetails(title='Inception', year=2010, director='Christopher Nolan', rating=8.8, actors=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Ellen Page', role='Ariadne'), Actor(name='Tom Hardy', role='Bane')], budget=160.0)

### TypeDict

In [19]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    title: Annotated[str, "The title of the movie"]
    year: Annotated[int, "The year the movie was released"]
    director: Annotated[str, "The director of the movie"]
    rating: Annotated[float, "The rating of the movie on a scale of 1 to 10"]
    
model_with_typedict_structure = model.with_structured_output(MovieDict)
model_with_typedict_structure.invoke("Provide the title, release year, director, and rating of the movie avengers.")


{'director': 'Joss Whedon', 'rating': 8, 'title': 'Avengers', 'year': 2012}

In [21]:
class Actor(TypedDict):
    name: str
    role: str
    
class MovieDetails(TypedDict):
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The rating of the movie on a scale of 1 to 10") 
    actors: list[Actor] = Field(..., description="A list of actors in the movie")
    budget: float | None = Field(None, description="The budget of the movie in millions of dollars")
    
model_with_structure = model.with_structured_output(MovieDetails)
model_with_structure.invoke("Provide the details about aavengers movie.")

{'actors': [{'name': 'Robert Downey Jr.', 'role': 'Iron Man'},
  {'name': 'Chris Evans', 'role': 'Captain America'},
  {'name': 'Mark Ruffalo', 'role': 'Hulk'},
  {'name': 'Chris Hemsworth', 'role': 'Thor'},
  {'name': 'Scarlett Johansson', 'role': 'Black Widow'},
  {'name': 'Jeremy Renner', 'role': 'Hawkeye'}],
 'budget': 220000000,
 'director': 'Joss Whedon',
 'rating': 8,
 'title': 'Avengers',
 'year': 2012}

In [22]:
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}

### Data Classes

In [23]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY","")

In [27]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    name: str = Field(..., description="The name of the contact")
    email: str = Field(..., description="The email address of the contact")
    phone: str = Field(..., description="The phone number of the contact")

agent = create_agent(
    model = "gpt-5", 
    response_format=ContactInfo
    )

result = agent.invoke({"messages":"Extract contact information from john doe, john@example.com, 553-915-215"})
result['structured_response']

ContactInfo(name='john doe', email='john@example.com', phone='553-915-215')

In [28]:
## TYPEDICT
from typing_extensions import TypedDict, Annotated
from langchain.agents import create_agent

class ContactInfo(TypedDict):
    name: str = Field(..., description="The name of the contact")
    email: str = Field(..., description="The email address of the contact")
    phone: str = Field(..., description="The phone number of the contact")

agent = create_agent(
    model = "gpt-5", 
    response_format=ContactInfo
    )

result = agent.invoke({"messages":"Extract contact information from john doe, john@example.com, 553-915-215"})
result['structured_response']

{'name': 'john doe', 'email': 'john@example.com', 'phone': '553-915-215'}

In [29]:
## Data Class

from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    name: str
    email: str
    phone: str
    
agent = create_agent(
    model = "gpt-5", 
    response_format=ContactInfo
)

result = agent.invoke({"messages":"Extract contact information from john doe, john@example.com, 553-915-215"})
result['structured_response']

ContactInfo(name='john doe', email='john@example.com', phone='553-915-215')